In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ml2021spring-hw2/sampleSubmission.csv
/kaggle/input/ml2021spring-hw2/timit_11/timit_11/train_11.npy
/kaggle/input/ml2021spring-hw2/timit_11/timit_11/test_11.npy
/kaggle/input/ml2021spring-hw2/timit_11/timit_11/train_label_11.npy


# Load Data

In [2]:
print('Loading data ...')
print(os.getcwd())

data_root='../input/ml2021spring-hw2/timit_11/timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
/kaggle/working
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


# DataSet

In [3]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)

# 超参数配置

In [4]:
VAL_RATIO = 0.1        # 验证集比例
BATCH_SIZE = 512

# training parameters
num_epoch = 50               # number of training epoch
learning_rate = 0.0001       # learning rate
weight_decay = 0     # 不知道为什么基本不能收敛
dropout = 0.1    # 多大概率丢失

# the path where checkpoint saved
model_path = './model.ckpt'

In [5]:
percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (1106938, 429)
Size of validation set: (122994, 429)


In [6]:


from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

In [7]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

80

# model

In [8]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(429, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        
        self.layer2 = nn.Linear(1024, 256)
        self.bn2 = nn.BatchNorm1d(256)
        
        self.layer3 = nn.Linear(256, 64)
        self.bn3 = nn.BatchNorm1d(64)
        
        self.out = nn.Linear(64, 39) 
        self.dropout = nn.Dropout(dropout)
        self.act_fn = nn.Sigmoid()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.dropout(x)
        x = self.bn1(x)
        x = self.act_fn(x)

        x = self.layer2(x)
        x = self.dropout(x)
        x = self.bn2(x)
        x = self.act_fn(x)

        x = self.layer3(x)
        x = self.dropout(x)
        x = self.bn3(x)
        x = self.act_fn(x)

        x = self.out(x)
        
        return x

# train

In [9]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
# fix random seed
# def same_seeds(seed):
#     torch.manual_seed(seed)
#     if torch.cuda.is_available():
#         torch.cuda.manual_seed(seed)
#         torch.cuda.manual_seed_all(seed)  
#     np.random.seed(seed)  
#     torch.backends.cudnn.benchmark = False
#     torch.backends.cudnn.deterministic = True

In [11]:
# fix random seed for reproducibility
# same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')


# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay = weight_decay)

DEVICE: cuda


In [12]:
# start training

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with val acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')

[001/050] Train Acc: 0.366569 Loss: 2.591440 | Val Acc: 0.442452 loss: 2.114810
saving model with val acc 0.442
[002/050] Train Acc: 0.478588 Loss: 1.916905 | Val Acc: 0.532546 loss: 1.655892
saving model with val acc 0.533
[003/050] Train Acc: 0.541635 Loss: 1.614705 | Val Acc: 0.593964 loss: 1.426979
saving model with val acc 0.594
[004/050] Train Acc: 0.577619 Loss: 1.452529 | Val Acc: 0.611314 loss: 1.301433
saving model with val acc 0.611
[005/050] Train Acc: 0.600430 Loss: 1.350428 | Val Acc: 0.637072 loss: 1.210813
saving model with val acc 0.637
[006/050] Train Acc: 0.616069 Loss: 1.282020 | Val Acc: 0.649812 loss: 1.146848
saving model with val acc 0.650
[007/050] Train Acc: 0.627174 Loss: 1.234129 | Val Acc: 0.660821 loss: 1.099274
saving model with val acc 0.661
[008/050] Train Acc: 0.635513 Loss: 1.197914 | Val Acc: 0.668911 loss: 1.066928
saving model with val acc 0.669
[009/050] Train Acc: 0.642452 Loss: 1.169371 | Val Acc: 0.675179 loss: 1.040947
saving model with val ac

In [13]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [14]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

In [15]:
predict

[36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 38,
 38,
 38,
 25,
 25,
 25,
 15,
 15,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 38,
 38,
 38,
 38,
 38,
 38,
 25,
 25,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 14,
 14,
 14,
 14,
 14,
 14,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 30,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 26,
 26,
 26,
 25,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 38,
 38,
 38,
 38,
 27,
 27,
 27,
 27,
 14,
 14,
 14,
 14,
 14,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,


In [16]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))
        